# Implementación Modelo J-PAS #

In [254]:
#Paquetes necesarios (seguramente sobra alguno)
import numpy as np
import math
import pandas as pd
import scipy as sp
import sympy
import scipy.integrate as integrate
import scipy.special as special
from mpmath import *
import numpy.testing as testing
import camb

Ejemplo de función vectorizada

In [255]:
# Your scalar function
def my_function(x):
    # Example function: square the input
    return x**2

# Vectorize the function
vectorized_function = np.vectorize(my_function)

# Input array
input_array = np.array([1, 2, 3, 4, 5])

# Apply the vectorized function to the array
result_array = vectorized_function(input_array)

In [256]:
vectorized_function(input_array)

array([ 1,  4,  9, 16, 25])

# Cosmología y bineado #

Parámetros cosmlógicos y arrays k,z:

In [257]:
#Valores de los parámetros cosmológicos en el fiducial.

#Parámetros cosmológicos directos:
hJPAS = 0.674; c = 2.99792458E5; OmegabJPASh2 = 0.02212; OmegaCDMJPASh2 = 0.1206; 
OmegakJPAS = 0; OmegamJPAS = 0.314170; AsJPAS = 2.09052E-9; nsJPAS = 0.9626; 
sigma8JPAS = 0.8120;

#Parámetros cosmológicos indirectos:
HJPAS = 1/(c/100); OmegabJPAS = OmegabJPASh2/hJPAS**2; OmegaCDMJPAS = OmegaCDMJPASh2/hJPAS**2;
OmegaLJPAS = 1 - OmegamJPAS;

#Límites y pasos de los arrays. Escalas en Mpc/h.
kminKArrayCompleto = 0.001;   kmaxKArrayCompleto = 2.4900;  pasoKArrayCompleto = 0.025;
zmin = 1.7;   zmax = 2.9;   pasoz = 0.2;

In [258]:
#Parámetros cosmológicos fuera del fiducial:
hJPASNoFid = hJPAS + hJPAS/100;
OmegabJPASh2NoFid = OmegabJPASh2 + OmegabJPASh2/100;
OmegaCDMJPASh2NoFid = OmegaCDMJPASh2 + OmegaCDMJPASh2/100; 
OmegamJPASNoFid = OmegamJPAS + OmegamJPAS/100;

#Parámetros cosmológicos indirectos fuera del fiducial:
OmegabJPASNoFid = OmegabJPASh2NoFid/hJPASNoFid**2; OmegaCDMJPASNoFid = OmegaCDMJPASh2NoFid/hJPASNoFid**2;
OmegaLJPASNoFid = 1 - OmegamJPASNoFid;

Bines de k y z

In [259]:
#Bines de k, completos y reducidos
KArrayCompleto = np.exp( np.arange(math.log(kminKArrayCompleto), math.log(kmaxKArrayCompleto), pasoKArrayCompleto) )
KArray = KArrayCompleto[range(121,245)]

In [260]:
#Bines de z:
za = np.arange(zmin, zmax+0.1, pasoz)

Llamadas a CAMB. Una para el fiducial, otra fuera de fiducial, para z=0 y para zupper y zlower

In [261]:
#Llamamos a CAMB para la cosmología fiducial
pars = camb.CAMBparams()
pars.set_cosmology(H0=100*hJPAS, ombh2=OmegabJPASh2, omch2=OmegaCDMJPASh2, mnu=0, omk=0, tau=0.06);
pars.set_matter_power(redshifts=za, kmax=KArrayCompleto[-1]);

#Aquí ya trabajamos con los resultados de CAMB 'results' y 'background':
results = camb.get_results(pars)
background = camb.get_background(pars, no_thermo=False)

Note: redshifts have been re-sorted (earliest first)


In [262]:
#Llamamos a otro CAMB leyendo datos fuera del fiducial
parsNoFid = camb.CAMBparams()
parsNoFid.set_cosmology(H0=100*hJPASNoFid, ombh2=OmegabJPASh2NoFid, omch2=OmegaCDMJPASh2NoFid, mnu=0, omk=0, tau=0.06);
parsNoFid.set_matter_power(redshifts=za, kmax=KArrayCompleto[-1]);
resultsNoFid = camb.get_results(parsNoFid)
backgroundNoFid = camb.get_background(parsNoFid, no_thermo=False)

Note: redshifts have been re-sorted (earliest first)


In [263]:
#Importamos otro CAMB leyendo datos a z=0
parsz0 = camb.CAMBparams()
parsz0.set_cosmology(H0=100*hJPAS, ombh2=OmegabJPASh2, omch2=OmegaCDMJPASh2, mnu=0, omk=0, tau=0.06);
parsz0.set_matter_power(redshifts=[0], kmax=KArrayCompleto[-1]);
resultsz0 = camb.get_results(parsz0)
backgroundz0 = camb.get_background(parsz0, no_thermo=False)

In [264]:
#Otro CAMB leyendo zaupper
parszupper = camb.CAMBparams()
parszupper.set_cosmology(H0=100*hJPAS, ombh2=OmegabJPASh2, omch2=OmegaCDMJPASh2, mnu=0, omk=0, tau=0.06);
parszupper.set_matter_power(redshifts=za+0.1, kmax=KArrayCompleto[-1]);
resultszupper = camb.get_results(parszupper)
backgroundzupper = camb.get_background(parszupper, no_thermo=False)

Note: redshifts have been re-sorted (earliest first)


In [265]:
#Otro CAMB leyendo zalower
parszlower = camb.CAMBparams()
parszlower.set_cosmology(H0=100*hJPAS, ombh2=OmegabJPASh2, omch2=OmegaCDMJPASh2, mnu=0, omk=0, tau=0.06);
parszlower.set_matter_power(redshifts=za-0.1, kmax=KArrayCompleto[-1]);
resultszlower = camb.get_results(parszlower)
backgroundzlower = camb.get_background(parszlower, no_thermo=False)

Note: redshifts have been re-sorted (earliest first)


# P(k) materia lineal #

P(k) de materia lineal desde CAMB

In [266]:
#Con este comando calculamos el P de materia (en za) en el array de k.
#Se guardan los 3 arrays a la vez ya que el output los da así:
kmatCAMB, zaCAMB, PmatCAMB = results.get_matter_power_spectrum(minkh=KArrayCompleto[0], maxkh=KArrayCompleto[-1], npoints = len(KArrayCompleto))

In [267]:
#P(k) materia fuera del fiducial:
kmatCAMBNoFid, zaCAMBNoFid, PmatCAMBNoFid = resultsNoFid.get_matter_power_spectrum(minkh=KArrayCompleto[0], maxkh=KArrayCompleto[-1], npoints = len(KArrayCompleto))

In [268]:
#P(k) materia a z=0:
kmatCAMBz0, zaCAMBz0, PmatCAMBz0 = resultsz0.get_matter_power_spectrum(minkh=KArrayCompleto[0], maxkh=KArrayCompleto[-1], npoints = len(KArrayCompleto))

Se interpola en k el P(k) a z = 0:

In [269]:
def PmatInterCAMB(k):
  return np.interp(k, kmatCAMBz0, PmatCAMBz0[0])

# Modelo de Kaiser #

Funciones E(z), f(z) y D(z):

In [270]:
#Leemos el valor de CAMB para Omegam, quedándonos solo con bariones y cdm:
Omegam = np.array(results.get_Omega('baryon'))+np.array(results.get_Omega('cdm'))

In [271]:
Ez = np.sqrt( Omegam*(1+za)**3+(1-Omegam) )

In [272]:
H = HJPAS * Ez

In [273]:
#Este es el sigma8 para los bines de z
sigma8Reves = np.array(results.get_sigma8())
sigma8 = sigma8Reves[::-1]

#sigma8 para z=0:
sigma8z0 = resultsz0.get_sigma8()[0]

In [274]:
#Llamando a fsigma8 de CAMB y dividiendo entre sigma8 obtenemos f(z).
#La ordenación se hace de z más alto a z más bajo y la cambiamos
fReves = np.array(results.get_fsigma8())/sigma8Reves
f = fReves[::-1]

In [275]:
#El growth factor D(z) no está directamente en CAMB, pero su forma es igual que la de sigmaR.
#Por tanto, usamos el sigmaR normalizado
DeReves = results.get_sigmaR(8)/resultsz0.get_sigmaR(8)[0]
De = DeReves[::-1]

Función A(z)

In [276]:
#Bias
def bJPAS(z):
  return 0.53+0.289*(1+z)**2

In [277]:
#Notar aquí que el sigma8 se pone por seguir la notación de Maroto, ya que luego cancela
A = De*bJPAS(za)*sigma8z0

Función R(z)

In [278]:
R = De*f*sigma8z0

Modelo Kaiser

In [279]:
def PKaiser(mu,k):
  return (A[0]+R[0]*mu**2)**2*PmatInterCAMB(k)/sigma8z0**2

# Fotometría #

In [280]:
#Valor del error fotométrico (3.6%)
DeltazJPAS = 0.00364236313918151

Función sigmar(z)

In [281]:
sigmar = DeltazJPAS*(1+za)/H

In [282]:
def PKaiserFotometria(mu,k):
  return PKaiser(mu,k)*np.exp(-(k*mu*sigmar[0])**2)

# Fingers of God #

Parámetro dispersión velocidades. Se obtiene una vez para el fiducial

In [283]:
sigmap = (1/(6*np.pi**2)*(De/1)**2*integrate.quad(lambda k: PmatInterCAMB(k), KArrayCompleto[0], KArrayCompleto[len(KArrayCompleto)-1])[0])**0.5

/var/folders/1h/0trg724n2fg2pn5j02gypgzc0000gn/T/ipykernel_44510/1298175680.py:1: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  sigmap = (1/(6*np.pi**2)*(De/1)**2*integrate.quad(lambda k: PmatInterCAMB(k), KArrayCompleto[0], KArrayCompleto[len(KArrayCompleto)-1])[0])**0.5


Cómputo factor Fingers of God

In [284]:
def FFog(mu,k):
    return 1/(1+(f[0]*k*mu*sigmap[0])**2)

P(k) de Kaiser con FFoG

In [285]:
def PKaiserFotometriaFoG(mu,k):
    return FFog(mu,k)*PKaiserFotometria(mu,k)

# Efecto AP #

In [286]:
#Distancia comóvil desde CAMB
Xi = results.comoving_radial_distance(za)*hJPAS

In [287]:
#Distancia angular. Multiplicamos por h para tener unidades h/Mpc:
DA = Xi/(1+za)

Factor de distorsión del AP:

In [288]:
FactorAP = DA**2*Ez/( DA**2*Ez )

Factor Q distorsión:

In [289]:
def Q(mu):
    return ((Ez[0]*Xi[0]*mu**2-Ez[0]**2*Xi[0]**2*(mu**2-1))**0.5/(Ez[0]*Xi[0]))

Mus y ks distorsionados

In [290]:
def muObs(mu):
    return mu*Ez[0]/(Ez[0]*Q(mu))

In [291]:
def kObs(mu,k):
    return Q(mu)*k

P de galaxias: Kaiser con fotometría, FoG y AP

In [292]:
#Vuelvo a escribir todos los factores ya que el efecto AP no se aplica por igual en todos
def Pg(mu,k):
    return FactorAP[0]*FFog(muObs(mu),kObs(mu,k))*(A[0]+R[0]*muObs(mu)**2)**2 * PmatInterCAMB(kObs(mu,k))/sigma8z0**2 *np.exp(-(k*mu*sigmar[0])**2)

# P galaxy monopole #

In [293]:
def Pgmonopole(k, axis = 0):
    mu = np.arange(-0.90, 0.90, 1/100)
    return 1/2 * integrate.trapz(mu, (lambda x: Pg(x, k))(mu), axis = axis)

In [294]:
PgmonopoleValores = np.zeros(len(KArray)-1)
for i in range(0, len(KArray)-1):
    PgmonopoleValores[i] = Pgmonopole(KArray[i])

# Covarianza #

Importación y lectura de las densidades

In [295]:
ImportacionDensityHighZ = [i.strip().split() for i in open("/Users/guillermo/Desktop/DensityHighZ.dat").readlines()]

In [296]:
DensityHighZ = np.zeros(len(ImportacionDensityHighZ));

In [297]:
for i in range(0, len(ImportacionDensityHighZ)):
  DensityHighZ[i] = ImportacionDensityHighZ[i][1]

Definición del volumen (requiere distancia angular con unidades y binear en zupper y zlower)

In [298]:
#Área del cielo
fsky = 0.2575;

In [299]:
#Bines de z por arriba y por abajo:
zaupper = za+(za[[1]]-za[[0]])/2;    zalower = za-(za[[1]]-za[[0]])/2;

In [300]:
#Distancia angular para los bines z upper y lower:
XiZaLower = resultszlower.comoving_radial_distance(zalower)*hJPAS
XiZaUpper = resultszupper.comoving_radial_distance(zaupper)*hJPAS

In [301]:
#Definición de volumen:
Vol = 4*np.pi*fsky/3*(XiZaUpper**3-XiZaLower**3)

Definición del número de modos (requiere arrays en kupper y klower):

In [302]:
#Bines de k por arriba y por abajo
KArrayUpper = np.zeros(len(KArray)); KArrayLower = np.zeros(len(KArray));

for i in range(0, len(KArray)-1):
  KArrayUpper[i] = KArray[i] + (KArray[i+1]-KArray[i])/2;   KArrayLower[i] = KArray[i] - (KArray[i+1]-KArray[i])/2;

KArrayUpper[len(KArray)-1] = KArrayUpper[len(KArray)-2];  KArrayLower[len(KArray)-1] = KArrayLower[len(KArray)-2];

In [303]:
#Número de modos. Depende de las variables k1 y k2, que debe correspodnerse kupper y klower
def Nk(k1,k2):
    return Vol[0] * (4*np.pi/3*(k1**3-k2**3))/((2*np.pi)**3)

In [304]:
#Evaluamos Nk para cada valor de nuestro array de k
NkEvaluado = np.zeros(len(KArray)-1)
for i in range(0, len(KArray)-1):
    NkEvaluado[i] = Nk(KArrayUpper[i],KArrayLower[i])

Definición de la covarianza

In [305]:
#Va a depender de k1 y k2. No me gusta:
def Cov(k,k1,k2):
    return 2 * (Pgmonopole(k) + 1/DensityHighZ[0])**2 / Nk(k1,k2)

In [306]:
#Evaluamos Cov para nuestros k
CovEvaluado = (PgmonopoleValores + 1/DensityHighZ[0])**2 / NkEvaluado

# Lectura de datos #

Importación de los de datos del P(k) monopolo que hemos generado

In [307]:
ImportacionDatosHighZ = [i.strip().split() for i in open("/Users/guillermo/Desktop/FicticioHighZArrayEnK.dat").readlines()]

In [308]:
DataPkzTotal = np.zeros(len(za));

In [309]:
Pk1 = np.zeros(len(ImportacionDatosHighZ)); Pk2 = np.zeros(len(ImportacionDatosHighZ));
Pk3 = np.zeros(len(ImportacionDatosHighZ)); Pk4 = np.zeros(len(ImportacionDatosHighZ));
Pk5 = np.zeros(len(ImportacionDatosHighZ)); Pk6 = np.zeros(len(ImportacionDatosHighZ));
Pk7 = np.zeros(len(ImportacionDatosHighZ));

In [310]:
for i in range(0, len(ImportacionDatosHighZ)-1):
  Pk1[i] = ImportacionDatosHighZ[i][2]; Pk2[i] = ImportacionDatosHighZ[i][3]; Pk3[i] = ImportacionDatosHighZ[i][4];
  Pk4[i] = ImportacionDatosHighZ[i][5]; Pk5[i] = ImportacionDatosHighZ[i][6]; Pk6[i] = ImportacionDatosHighZ[i][7];
  Pk7[i] = ImportacionDatosHighZ[i][8]

In [311]:
DataPkzTotal = np.array([Pk1,Pk2,Pk3,Pk4,Pk5,Pk6,Pk7])

# Likelihood #

Bineamos el P(k) para cuando entre al likelihood:

In [326]:
PgBineadoz1 = np.zeros(len(KArray)); PgBineadoz2 = np.zeros(len(KArray));
PgBineadoz3 = np.zeros(len(KArray)); PgBineadoz4 = np.zeros(len(KArray));
PgBineadoz5 = np.zeros(len(KArray)); PgBineadoz6 = np.zeros(len(KArray));
PgBineadoz7 = np.zeros(len(KArray));

In [327]:
PgBineado = np.array([PgBineadoz1,PgBineadoz2,PgBineadoz3,PgBineadoz4,PgBineadoz5,PgBineadoz6,PgBineadoz7])

In [328]:
for i in range(0, len(KArray)-1):
    PgBineado[0][i] = PgmonopoleCAMBValores[i]

Bineamos la covarianza para cuando entre al likelihood

In [315]:
CovBineadoz1 = np.zeros(len(KArray)); CovBineadoz2 = np.zeros(len(KArray));
CovBineadoz3 = np.zeros(len(KArray)); CovBineadoz4 = np.zeros(len(KArray));
CovBineadoz5 = np.zeros(len(KArray)); CovBineadoz6 = np.zeros(len(KArray));
CovBineadoz7 = np.zeros(len(KArray));

In [316]:
CovBineado = np.array([CovBineadoz1,CovBineadoz2,CovBineadoz3,CovBineadoz4,CovBineadoz5,CovBineadoz6,CovBineadoz7])

In [317]:
for i in range(0, len(KArray)-1):
    CovBineado[0][i] = CovEvaluado[i]

Construimos el likelihood:

In [318]:
def lnlikeSinSumar(j,i):
    return (PgBineado[j][i]-DataPkzTotal[j][i])**2 * 1/CovBineado[j][i] + np.log(CovBineado[j][i])

índices en los que queremos evaluar el likelihood

In [319]:
IndicesLikelihood = np.arange(0,len(KArray)-40,1)

In [336]:
#Likelihood sumando los índices
lnlike = np.sum(lnlikeSinSumar(0,IndicesLikelihood))
lnlike

70717.07091017577